# Ejercicio 1.1
## Jorge Pablo Ávila Gómez

## Ejercicio 1.1: Contador de clientes valorados por países.
Partiendo de los ficheros de datos de países y clientes y del código visto en el
ejemplo mrjob-join, mejora dicho código para que el programa devuelva
cuántos clientes con valoración “bueno” hay en cada país. En concreto, la salida
del programa MapReduce debe ser un fichero con el contenido que se muestra
en la Figura 1 . Este ejercicio tiene una puntuación de hasta 3 puntos sobre 10.

### Diseño del programa MapReduce
#### Limpieza de los datos
En la primera parte del ejercicio se observan los documentos countries.csv y clientes.csv.
En particular se observa que el documento countries.csv tiene cabezera (Name,Code) y que algunos paises tiene una coma dentro de su nombre lo cual puede crear problemas a la hora de separar las columnas. por ejemplo: `"Bolivia, Plurinational State of"`

Se elimina la cabezera del documento y se limpian las columnas quitando la coma y el entrecomillado:

`"Bolivia, Plurinational State of" -> Plurinational State of Bolivia`
#### Diseño MapReduce
El ejercicio se ha planteado usando un paso Map y un paso Reduce.
- Map. El map se usa para leer los dos documentos y extraer la información necesaria.
Se necesita extraer la columna nombre del pais del documentro countries y la columna valoración == 'bueno' del documento clients. Las dos tablas se encuentran enlazadas por la columna country2digit.

Para el documento countries se separa en dos columnas, nombre del pais y country2digit. Se envia como clave la columna country2digit y como valor una lista formada por la letra 'A' y el nombre del pais.

Para el documento clients se separa en tres columnas, nombre del cliente, valoración y country2digit. Aquí es importante que filtremos las columnas en las que la valoración es 'bueno'. Enviamos como clave country2digit, que es la columna que nos sirve para relacionar las dos tablas, y como valor enviamos una lista formada por la letra 'B' y como segundo elemento el número 1. Indicando que para ese pais hemos encontrado al menos 1 valoración buena.

Se utiliza las etiquetas 'A' y 'B' junto con la opción: 
```python 
SORT_VALUES = True```
Para que al reducer los valores llegen ordenados. Como sabemos, habrá un reducer para cada country2digit, es decir para cada país. Y los valores vendrán ordenados primero con la etiqueta 'A', solo habrá una y corresponde al documento countries con el nombre completo del pais, y luego todos los valores con la etiqueta 'B', procedentes del documento clients indicando que se ha encontrado una valoración buena.

- Reduce. La función reducer va a contar el numero de valoraciones positivas que le llega de cada pais.

Por tanto para cada country2digit primero recibirá una lista con la etiqueda 'A' y el nombre completo del pais. Luego recibirá un número de valores con la etiqueta 'B'. El reducer va sumando cuantas 'B' le llegan correspodiendose cada una con una valoración buena.

Para cada country2digit devuelve una tuple con dos valores el nombre del pais y la suma de las etiquetas. 

In [1]:
# Se crea la carpeta ejercicio101 donde se van a almacenar los documentos que se generen
! mkdir -p tp1/ejercicio101
import os
os.chdir("/media/notebooks/tp1/ejercicio101")
! pwd

/media/notebooks/tp1/ejercicio101


In [2]:
# Los ficheros countries.csv y clients.csv deben estar descargados en la carpeta /media/notebooks/tp1/ejercicio101
! cp /media/notebooks/countries.csv /media/notebooks/tp1/ejercicio101/
! cp /media/notebooks/clients.csv /media/notebooks/tp1/ejercicio101/

In [3]:
! head countries.csv

In [4]:
## Limpieza de countries.csv

with open("countries.csv",'r') as f, open("countries_cleaned.csv",'w') as f1:
    next(f) # salta el encabezado
    for line in f:
        if '''"''' in line: # Los paises con el problema de la coma estan entrecomillados
            loc = line.find(',') # Elimina la primera coma para no confundir con la separacion entre columnas
            new_line = line[loc+2:-5] + ' ' + line[1:loc] + line[-4:] # Reordena el nombre del pais
            f1.write(new_line)
            print(line)
            print(new_line)
        else:
            f1.write(line)

"Bolivia, Plurinational State of",BO

Plurinational State of Bolivia,BO

"Bonaire, Sint Eustatius and Saba",BQ

Sint Eustatius and Saba Bonaire,BQ

"Congo, the Democratic Republic of the",CD

the Democratic Republic of the Congo,CD

"Iran, Islamic Republic of",IR

Islamic Republic of Iran,IR

"Korea, Democratic People's Republic of",KP

Democratic People's Republic of Korea,KP

"Korea, Republic of",KR

Republic of Korea,KR

"Macedonia, the Former Yugoslav Republic of",MK

the Former Yugoslav Republic of Macedonia,MK

"Micronesia, Federated States of",FM

Federated States of Micronesia,FM

"Moldova, Republic of",MD

Republic of Moldova,MD

"Palestine, State of",PS

State of Palestine,PS

"Saint Helena, Ascension and Tristan da Cunha",SH

Ascension and Tristan da Cunha Saint Helena,SH

"Taiwan, Province of China",TW

Province of China Taiwan,TW

"Tanzania, United Republic of",TZ

United Republic of Tanzania,TZ

"Venezuela, Bolivarian Republic of",VE

Bolivarian Republic of Venezuela,VE



In [5]:
! head countries_cleaned.csv

Afghanistan,AF
Åland Islands,AX
Albania,AL
Algeria,DZ
American Samoa,AS
Andorra,AD
Angola,AO
Anguilla,AI
Antarctica,AQ
Antigua and Barbuda,AG


In [6]:
! head clients.csv

Bertram Pearcy  ,bueno,SO
Steven Ulman  ,regular,ZA
Enid Follansbee  ,malo,GS
Candie Jacko  ,malo,SS
Alana Zufelt  ,regular,ES
Craig Pinkett  ,malo,LK
Carson Levey  ,bueno,GU
Reanna Calabrese  ,regular,GT
Elliott Kosak  ,malo,GG
Yuette Steinman  ,bueno,GN


In [7]:
%%writefile mrjob-ejercicio101.py

from mrjob.job import MRJob

class MRJoin(MRJob):

    SORT_VALUES = True

    def mapper(self, _, line):
        splits = line.rstrip("\n").split(",")
        if len(splits) == 2: # datos de paises
            symbol = 'A'     # diferencia datos de paises de clientes
            country2digit = splits[1]
            yield country2digit, [symbol, splits[0]] #valor es el simbolo A y el nombre del pais completo
        else: #  datos de personas
            symbol = 'B' 
            country2digit = splits[2]
            if splits[1] == 'bueno': # seleccionamos solo los clientes con una valoracion buena
                yield country2digit, [symbol, 1] # enviamos solo un valor cuando hemos encontrado una val buena 
        
    def reducer(self, key, values):
        for value in values:
            if value[0] == 'A': # dato de pais
                country = value[1] # nombre del pais
                counter = 0
            if value[0] == 'B': # dato de persona
                counter +=1 # sumamos 1 por cada valoración buena
        else:
            if counter > 0: # solo enviamos resultados hay al menos 1 valoración buena
                yield country, counter
                
if __name__ == '__main__':
    MRJoin.run()

Overwriting mrjob-ejercicio101.py


#### Primero ejecutamos el código en local:

In [8]:
! python mrjob-ejercicio101.py /media/notebooks/tp1/ejercicio101/countries_cleaned.csv  \
/media/notebooks/tp1/ejercicio101/clients.csv > ouputlocal

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/mrjob-ejercicio101.root.20201030.165619.968030
Running step 1 of 1...
job output is in /tmp/mrjob-ejercicio101.root.20201030.165619.968030/output
Streaming final output from /tmp/mrjob-ejercicio101.root.20201030.165619.968030/output...
Removing temp directory /tmp/mrjob-ejercicio101.root.20201030.165619.968030...


In [9]:
! cat ouputlocal

"Spain"	3
"Guinea"	1
"South Georgia and the South Sandwich Islands"	1
"Guam"	3
"Canada"	1
"Somalia"	1
"South Sudan"	1
"Turkey"	1
"United States"	1
"South Africa"	1
"Portugal"	1
"Qatar"	1


#### Ejecutamos el código en hadoop:

In [10]:
! hdfs dfs -mkdir /tmp/mrjoin/ejercicio101
! hdfs dfs -put /media/notebooks/tp1/ejercicio101/countries_cleaned.csv  /tmp/mrjoin/ejercicio101
! hdfs dfs -put /media/notebooks/tp1/ejercicio101/clients.csv  /tmp/mrjoin/ejercicio101

mkdir: `/tmp/mrjoin/ejercicio101': File exists
put: `/tmp/mrjoin/ejercicio101/countries_cleaned.csv': File exists
put: `/tmp/mrjoin/ejercicio101/clients.csv': File exists


In [11]:
! hdfs dfs -rm /tmp/carpeta/mrjob-ejercicio101/*
! hdfs dfs -rmdir /tmp/carpeta/mrjob-ejercicio101

Deleted /tmp/carpeta/mrjob-ejercicio101/_SUCCESS
Deleted /tmp/carpeta/mrjob-ejercicio101/part-00000


In [12]:
! python mrjob-ejercicio101.py hdfs:///tmp/mrjoin/ejercicio101/* -r hadoop --python-bin /opt/anaconda/bin/python3.7 \
--output-dir hdfs:///tmp/carpeta/mrjob-ejercicio101

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /usr/lib/hadoop/bin...
Found hadoop binary: /usr/lib/hadoop/bin/hadoop
Using Hadoop version 2.6.0
Looking for Hadoop streaming jar in /usr/lib/hadoop...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Creating temp directory /tmp/mrjob-ejercicio101.root.20201030.165632.519590
uploading working dir files to hdfs:///user/root/tmp/mrjob/mrjob-ejercicio101.root.20201030.165632.519590/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/mrjob-ejercicio101.root.20201030.165632.519590/files/
Running step 1 of 1...
  packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob1409532757746871196.jar tmpDir=null
  Connecting to ResourceManager at yarnmaster/172.19.0.4:8032
  Connecting to ResourceManager at yarnmaster/172.19.0.4:8032

In [13]:
! hdfs dfs -tail /tmp/carpeta/mrjob-ejercicio101/part-00000

"Canada"	1
"Spain"	3
"Guinea"	1
"South Georgia and the South Sandwich Islands"	1
"Guam"	3
"Portugal"	1
"Qatar"	1
"Somalia"	1
"South Sudan"	1
"Turkey"	1
"United States"	1
"South Africa"	1
